In [3]:
from hmmaudio.utils import load_all_data
from hmmlearn import hmm
from hmmaudio.eval import evaluate_models
import numpy as np

# Emotion Recognition using Existing HMM packages

This notebook demonstrates how to use our preprocessing pipeline and existing HMM packages to train and test an emotion recognition model.

In [4]:
TRAIN_DATA_PATH = "./data/train"  # Path to training data

# Load training data with delta and delta-second features
train_features, train_files = load_all_data(
    TRAIN_DATA_PATH,
    include_mfcc=True,
    include_delta=True,
    include_delta2=True,
    num_cepstral=13,
    target_frames=128
)

Processing Happy:   0%|          | 0/1271 [00:00<?, ?it/s]

Processing Happy: 100%|██████████| 1271/1271 [00:10<00:00, 119.86it/s]


Happy: Loaded 1271 files


Processing Sad: 100%|██████████| 1271/1271 [00:10<00:00, 118.32it/s]


Sad: Loaded 1271 files


...

Neutral: Loaded 1087 files


Processing Anger: 100%|██████████| 1271/1271 [00:10<00:00, 122.27it/s]


Anger: Loaded 1271 files


Processing Disgust: 100%|██████████| 1271/1271 [00:10<00:00, 120.24it/s]

Disgust: Loaded 1271 files


In [6]:
# Ensure the training data is loaded
def train_hmm(train_features, n_states):
    """
    Train Gaussian HMMs for each emotion using the provided features.

    Parameters:
    - train_features: Dictionary of lists of training features for each emotion.
    - n_states: Number of hidden states in the HMM.

    Returns:
    - models: Dictionary of trained HMM models for each emotion.
    """
    models = {}
    for emotion, features_list in train_features.items():
        # Concatenate all sequences and get their lengths
        X = np.concatenate(features_list)
        lengths = [features.shape[0] for features in features_list]
        print(f"Training HMM for {emotion} with {len(lengths)} sequences")
        # Initialize and train HMM
        model = hmm.GaussianHMM(n_components=n_states, 
                               covariance_type="diag", 
                               n_iter=10)
        # Fit the model to the data
        model.fit(X, lengths)
        models[emotion] = model
    return models

# Define HMM parameters
n_states = 5  # Number of hidden states
n_symbols = 13  # Number of MFCC features (including delta and delta-second)

# Train Gaussian HMMs for each emotion
hmm_models = train_hmm(train_features, n_states)

Training HMM for Happy with 1271 sequences
Training HMM for Sad with 1271 sequences
Training HMM for Fear with 1271 sequences
Training HMM for Neutral with 1087 sequences
Training HMM for Anger with 1271 sequences
Training HMM for Disgust with 1271 sequences


In [8]:
train_accuracy, train_cm, _, _, _ =evaluate_models(hmm_models, train_features, train_files)
print(f"Accuracy: {train_accuracy:.2f}")
print("Confusion Matrix:")
print(train_cm)

Evaluating HMMs...


Evaluating Disgust: 100%|██████████| 1271/1271 [00:01<00:00, 691.76it/s]

Evaluation complete.
Accuracy: 0.41
Confusion Matrix:
[[319 123 202 150 402  75]
 [ 26 789  97 183  44 132]
 [ 91 372 349  99 277  83]
 [ 90 278 112 346  73 188]
 [129  14  76  47 949  56]
 [134 293 171 180 224 269]]
